In [3]:
!pip -q install requests pdfplumber langchain langchain-community faiss-cpu sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 2.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4

In [4]:
import os
import re
import requests
import pdfplumber
from datetime import datetime

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter



In [5]:
import os
os.environ["OPENWEATHER_API_KEY"] = "Your API key"
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")

In [6]:
def get_current_weather(city: str) -> dict:
    url = "https://api.openweathermap.org/data/2.5/weather"
    params = {"q": city, "appid": OPENWEATHER_API_KEY, "units": "metric"}

    r = requests.get(url, params=params, timeout=20)
    data = r.json()

    if data.get("cod") != 200:
        raise ValueError(f"OpenWeather Current error: {data.get('message')}")

    return {
        "city": city,
        "temp": data["main"]["temp"],
        "humidity": data["main"]["humidity"],
        "pressure": data["main"]["pressure"],
        "wind_speed": data.get("wind", {}).get("speed", 0),
        "wind_deg": data.get("wind", {}).get("deg"),
        "clouds": data.get("clouds", {}).get("all"),
        "rain_1h": data.get("rain", {}).get("1h", 0),
        "description": data["weather"][0]["description"],
    }


def get_24h_forecast_summary(city: str) -> dict:
    url = "https://api.openweathermap.org/data/2.5/forecast"
    params = {"q": city, "appid": OPENWEATHER_API_KEY, "units": "metric"}

    r = requests.get(url, params=params, timeout=20)
    data = r.json()

    if data.get("cod") != "200":
        raise ValueError(f"OpenWeather Forecast error: {data.get('message')}")

    next_items = data["list"][:8]  # 8*3h = 24h

    temps = [x["main"]["temp"] for x in next_items]
    rains = [x.get("rain", {}).get("3h", 0) for x in next_items]
    winds = [x.get("wind", {}).get("speed", 0) for x in next_items]

    return {
        "temp_min_24h": round(min(temps), 2),
        "temp_max_24h": round(max(temps), 2),
        "rain_total_24h": round(sum(rains), 2),
        "wind_max_24h": round(max(winds), 2),
    }


def get_24h_forecast_summary(city: str) -> dict:
    """
    Uses OpenWeather 5-day/3-hour forecast endpoint.
    Returns next 24 hour summary.
    """
    url = "https://api.openweathermap.org/data/2.5/forecast"
    params = {"q": city, "appid": OPENWEATHER_API_KEY, "units": "metric"}

    r = requests.get(url, params=params, timeout=20)
    data = r.json()

    if data.get("cod") != "200":
        raise ValueError(f"OpenWeather Forecast error: {data.get('message')}")

    next_items = data["list"][:8]  # 8*3h = 24h

    temps = [x["main"]["temp"] for x in next_items]
    rains = [x.get("rain", {}).get("3h", 0) for x in next_items]
    winds = [x.get("wind", {}).get("speed", 0) for x in next_items]

    return {
        "temp_min_24h": round(min(temps), 2),
        "temp_max_24h": round(max(temps), 2),
        "rain_total_24h": round(sum(rains), 2),
        "wind_max_24h": round(max(winds), 2),
    }


In [7]:
def geocode_city_openweather(city: str) -> dict:
    url = f"http://api.openweathermap.org/geo/1.0/direct?q={city}&limit=1&appid={OPENWEATHER_API_KEY}"
    data = requests.get(url, timeout=20).json()

    if not data:
        raise ValueError(f"City not found in OpenWeather Geocoding: {city}")

    obj = data[0]
    return {
        "city": obj.get("name", city),
        "state": obj.get("state"),
        "country": obj.get("country"),
        "lat": obj["lat"],
        "lon": obj["lon"]
    }


def reverse_geocode_district(lat: float, lon: float):
    url = "https://nominatim.openstreetmap.org/reverse"
    params = {"lat": lat, "lon": lon, "format": "json", "zoom": 10, "addressdetails": 1}
    headers = {"User-Agent": "AgroAdvisoryBot/1.0"}

    data = requests.get(url, params=params, headers=headers, timeout=20).json()
    address = data.get("address", {})

    district = (
        address.get("state_district")
        or address.get("county")
        or address.get("district")
        or address.get("region")
    )

    if not district:
        return None

    return district.replace(" District", "").strip()


In [8]:
def download_imd_district_bulletin(state: str, district: str, save_dir="imd_cache"):
    os.makedirs(save_dir, exist_ok=True)

    url = "https://imdagrimet.gov.in/Services/DistrictBulletin.php"
    params = {"state": state, "district": district, "language": "English"}

    r = requests.get(url, params=params, timeout=30)
    content_type = (r.headers.get("Content-Type") or "").lower()

    if "pdf" in content_type:
        today = datetime.now().strftime("%Y-%m-%d")
        safe_state = re.sub(r"[^a-zA-Z0-9_-]+", "_", state)
        safe_dist = re.sub(r"[^a-zA-Z0-9_-]+", "_", district)
        pdf_path = os.path.join(save_dir, f"{safe_state}_{safe_dist}_{today}.pdf")

        with open(pdf_path, "wb") as f:
            f.write(r.content)

        return pdf_path, r.url, "DISTRICT"

    return None, r.url, None


def download_imd_state_bulletin(state: str, save_dir="imd_cache"):
    os.makedirs(save_dir, exist_ok=True)

    url = "https://imdagrimet.gov.in/Services/StateBulletin.php"
    params = {"state": state, "language": "English"}

    r = requests.get(url, params=params, timeout=30)
    content_type = (r.headers.get("Content-Type") or "").lower()

    if "pdf" not in content_type:
        raise ValueError(f"State bulletin did not return PDF. URL={r.url}\nPreview={r.text[:200]}")

    today = datetime.now().strftime("%Y-%m-%d")
    safe_state = re.sub(r"[^a-zA-Z0-9_-]+", "_", state)
    pdf_path = os.path.join(save_dir, f"{safe_state}_STATE_{today}.pdf")

    with open(pdf_path, "wb") as f:
        f.write(r.content)

    return pdf_path, r.url, "STATE"


In [9]:
def extract_pdf_text(pdf_path: str, max_pages=10) -> str:
    """
    Extract text from first max_pages pages of PDF using pdfplumber.
    (We do NOT parse tables; only advisory text.)
    """
    texts = []
    with pdfplumber.open(pdf_path) as pdf:
        n = min(len(pdf.pages), max_pages)
        for i in range(n):
            txt = pdf.pages[i].extract_text() or ""
            txt = re.sub(r"\s+\n", "\n", txt)
            texts.append(txt)
    return "\n\n".join(texts)


In [10]:
def build_vectorstore_from_text(text: str) -> FAISS:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=900,
        chunk_overlap=180,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    chunks = splitter.split_text(text)
    docs = [Document(page_content=c) for c in chunks]

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = FAISS.from_documents(docs, embeddings)
    return vectorstore


In [11]:
def retrieve_relevant_docs(vectorstore: FAISS, weather: dict, forecast: dict, k=5):
    """
    Retrieval query uses live weather.
    """
    query = (
        f"weather advisory irrigation guidance wind spraying rainfall humidity disease pest "
        f"temperature {weather['temp']} humidity {weather['humidity']} wind {weather['wind_speed']} "
        f"rain {weather['rain_1h']} forecast_rain {forecast['rain_total_24h']}"
    )

    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": k})
    docs = retriever.invoke(query)
    return query, docs


In [12]:
def clean_and_merge_points(points):
    cleaned = []
    buffer = ""

    for p in points:
        p = re.sub(r"[▪■◾◆]+", "", p)
        p = re.sub(r"\s+", " ", p).strip()

        if not p:
            continue

        # If looks like continuation line
        if p.startswith(")") or p.startswith("Wilt)") or p.startswith("Rot)") or p.startswith("Mildew)") or p.startswith("Blight)"):
            buffer = (buffer + " " + p).strip()
            continue

        if buffer:
            cleaned.append(buffer)
            buffer = ""

        buffer = p

        # If complete sentence
        if buffer.endswith(".") or len(buffer) > 140:
            cleaned.append(buffer)
            buffer = ""

    if buffer:
        cleaned.append(buffer)

    # Remove duplicates
    final = []
    for x in cleaned:
        if x not in final and len(x) > 20:
            final.append(x)

    return final


In [13]:
def categorize_advisory_points(retrieved_docs, max_points_per_category=5):
    categories = {
        "Weather warnings": [],
        "Crop pest/disease alert": [],
        "Irrigation guidance": [],
        "Spraying / fertilizer guidance": [],
        "Harvest / storage / field operations": []
    }

    rules = {
        "Weather warnings": ["warning", "alert", "thunderstorm", "hailstorm", "heavy rain", "strong wind", "fog", "cold wave", "heat wave", "nil"],
        "Crop pest/disease alert": ["pest", "disease", "mite", "thrips", "aphid", "bollworm", "rust", "blight", "wilt", "rot", "fungal", "incidence", "mildew", "worm"],
        "Irrigation guidance": ["irrigation", "irrigate", "watering", "drainage", "waterlogging", "reduce irrigation", "light irrigation"],
        "Spraying / fertilizer guidance": ["spray", "spraying", "pesticide", "insecticide", "fungicide", "herbicide", "fertilizer", "dose", "@", "apply"],
        "Harvest / storage / field operations": ["sowing", "harvesting", "threshing", "weeding", "interculture", "plough", "storage", "nursery", "transplant"]
    }

    heading_patterns = [
        r"^A\.\d+(\.\d+)?",
        r"^WEATHER WARNINGS",
        r"^Likely impacts",
        r"^Realized Rainfall",
        r"^Bulletin Number",
        r"^India Meteorological",
        r"^Government of India",
    ]

    action_words = ["spray", "apply", "control", "monitor", "avoid", "drench", "irrigation", "ensure", "minimize", "prevent", "trap"]

    def is_heading(line):
        l = line.strip()
        if len(l) < 18:
            return True
        for pat in heading_patterns:
            if re.search(pat, l, re.IGNORECASE):
                return True
        if l.endswith(":") and not any(w in l.lower() for w in action_words):
            return True
        return False

    def looks_actionable(line):
        low = line.lower()
        return any(w in low for w in action_words)

    def add(cat, line):
        if line and line not in categories[cat]:
            categories[cat].append(line)

    raw = []
    for doc in retrieved_docs:
        parts = re.split(r"[•\n]", doc.page_content)
        for p in parts:
            p = re.sub(r"\s+", " ", p).strip()
            if p:
                raw.append(p)

    # Merge broken lines quickly
    merged = []
    for line in raw:
        if not merged:
            merged.append(line)
            continue
        prev = merged[-1]
        if len(prev) < 70 and not prev.endswith("."):
            merged[-1] = (prev + " " + line).strip()
        else:
            merged.append(line)

    for line in merged:
        if is_heading(line):
            continue
        if not looks_actionable(line):
            continue

        low = line.lower()

        # spray category forced if it contains spray keywords
        if any(k in low for k in rules["Spraying / fertilizer guidance"]):
            add("Spraying / fertilizer guidance", line)
            continue

        if any(k in low for k in rules["Irrigation guidance"]):
            add("Irrigation guidance", line)
            continue

        if any(k in low for k in rules["Crop pest/disease alert"]):
            add("Crop pest/disease alert", line)
            continue

        if any(k in low for k in rules["Harvest / storage / field operations"]):
            add("Harvest / storage / field operations", line)
            continue

        if any(k in low for k in rules["Weather warnings"]):
            add("Weather warnings", line)
            continue

    # Trim
    for c in categories:
        categories[c] = categories[c][:max_points_per_category]

    return categories


In [14]:
def filter_by_crop(categorized, crop=None):
    if not crop:
        return categorized

    crop = crop.lower()
    filtered = {}
    for section, pts in categorized.items():
        keep = []
        for p in pts:
            if crop in p.lower():
                keep.append(p)
        filtered[section] = keep
    return filtered


In [15]:
def add_weather_based_advice(weather: dict, forecast: dict, categories: dict):
    temp = weather["temp"]
    hum = weather["humidity"]
    wind = weather["wind_speed"]
    rain_24 = forecast["rain_total_24h"]

    # warnings
    if rain_24 == 0:
        categories["Weather warnings"].insert(0, "No rainfall expected in next 24 hours.")
    if wind >= 6:
        categories["Weather warnings"].insert(0, f"High wind ({wind} m/s): Avoid spraying during peak wind hours.")

    # irrigation
    if hum < 45 and rain_24 == 0:
        categories["Irrigation guidance"].insert(0, "Low humidity + no rain: Increase irrigation frequency and conserve moisture using mulching.")
    if rain_24 >= 5:
        categories["Irrigation guidance"].insert(0, "Rain expected: Reduce irrigation, clear drainage channels to prevent waterlogging.")

    # spray
    if wind >= 6:
        categories["Spraying / fertilizer guidance"].insert(0, "Avoid spraying during windy hours; prefer early morning.")
    if rain_24 >= 3:
        categories["Spraying / fertilizer guidance"].insert(0, "Avoid spraying before rain; chemical may wash off.")

    # field operations
    if rain_24 == 0:
        categories["Harvest / storage / field operations"].insert(0, "No rain expected: Suitable window for field operations (weeding/interculture/harvesting).")

    # crop disease note
    if hum >= 75:
        categories["Crop pest/disease alert"].insert(0, f"High humidity ({hum}%): Higher fungal disease risk — monitor leaf spot/powdery mildew.")

    # Limit each category
    for k in categories:
        categories[k] = clean_and_merge_points(categories[k])[:6]

    return categories


In [16]:
def rank_top_advisories(categorized, weather, forecast, top_n=5):
    """
    Prioritize important advisories depending on weather situation.
    Returns list of (priority_score, category, advice_line)
    """
    temp = weather["temp"]
    hum = weather["humidity"]
    wind = weather["wind_speed"]
    rain_24 = forecast["rain_total_24h"]

    ranked = []

    def score_line(cat, line):
        s = 0
        low = line.lower()

        # category priority
        if cat == "Weather warnings":
            s += 5
        if cat == "Crop pest/disease alert":
            s += 4
        if cat == "Spraying / fertilizer guidance":
            s += 3
        if cat == "Irrigation guidance":
            s += 3
        if cat == "Harvest / storage / field operations":
            s += 2

        # weather match boosts
        if wind >= 6 and "spray" in low:
            s += 4
        if rain_24 >= 5 and ("drainage" in low or "waterlogging" in low):
            s += 4
        if hum < 45 and ("irrigation" in low or "watering" in low):
            s += 3
        if hum >= 75 and ("fungal" in low or "mildew" in low):
            s += 3

        return s

    for cat, pts in categorized.items():
        for p in pts:
            ranked.append((score_line(cat, p), cat, p))

    ranked.sort(reverse=True, key=lambda x: x[0])
    return ranked[:top_n]


In [17]:
def extract_bulletin_metadata(text: str):
    """
    Try to extract Bulletin Number and Date range from IMD PDF text.
    """
    meta = {}

    # Bulletin Number (example: Bulletin Number: 05/2026)
    m = re.search(r"Bulletin\s*Number\s*[:\-]?\s*([0-9/]+)", text, re.IGNORECASE)
    if m:
        meta["bulletin_number"] = m.group(1)

    # Date range in forecast lines (example: FROM 17-01-2026 to 21-01-2026)
    d = re.search(r"FROM\s+(\d{2}-\d{2}-\d{4})\s+to\s+(\d{2}-\d{2}-\d{4})", text, re.IGNORECASE)
    if d:
        meta["forecast_date_from"] = d.group(1)
        meta["forecast_date_to"] = d.group(2)

    return meta


def build_proof_section(weather, forecast, bulletin_url, bulletin_meta, docs):
    """
    Builds a structured proof object for your results.
    """
    proof = {}

    proof["weather_proof"] = {
        "source": "OpenWeatherMap API",
        "live_values_used": {
            "temp": weather["temp"],
            "humidity": weather["humidity"],
            "wind_speed": weather["wind_speed"],
            "rain_1h": weather["rain_1h"],
            "forecast_rain_24h": forecast["rain_total_24h"]
        }
    }

    proof["bulletin_proof"] = {
        "source": "IMD GKMS / IMD Agrimet official bulletin PDF",
        "bulletin_url": bulletin_url,
        "bulletin_number": bulletin_meta.get("bulletin_number"),
        "forecast_date_from": bulletin_meta.get("forecast_date_from"),
        "forecast_date_to": bulletin_meta.get("forecast_date_to"),
    }

    proof["rag_evidence"] = []
    for i, d in enumerate(docs, 1):
        snip = re.sub(r"\s+", " ", d.page_content.strip())
        proof["rag_evidence"].append({
            "rank": i,
            "snippet": snip[:350]
        })

    return proof


In [22]:
def smart_farm_advisory(city: str, crop: str=None):
    line = "=" * 80

    print(line)
    print(f"🌾 SMART FARM ADVISORY SYSTEM | CITY: {city}")
    print(line)

    # -------------- WEATHER ----------------
    weather = get_current_weather(city)
    forecast = get_24h_forecast_summary(city)

    print("\n🌦 REAL-TIME WEATHER (OpenWeather)")
    print(f"  Temperature : {weather.get('temp', 'NA')} °C")
    print(f"  Humidity    : {weather.get('humidity', 'NA')} %")
    print(f"  Wind Speed  : {weather.get('wind_speed', 'NA')} m/s")
    print(f"  Rain (1h)   : {weather.get('rain_1h', 0)} mm")
    print(f"  Condition   : {weather.get('description', 'NA')}")

    print("\n⏳ NEXT 24H FORECAST SUMMARY")
    print(f"  Temp Range  : {forecast.get('temp_min_24h','NA')}°C to {forecast.get('temp_max_24h','NA')}°C")
    print(f"  Rain Total  : {forecast.get('rain_total_24h',0)} mm")
    print(f"  Wind Max    : {forecast.get('wind_max_24h','NA')} m/s")

    # -------------- LOCATION --------------
    loc = geocode_city_openweather(city)
    state = loc.get("state")
    district = reverse_geocode_district(loc["lat"], loc["lon"])

    print("\n📍 LOCATION MAPPING")
    print(f"  State   : {state}")
    print(f"  District: {district}")

    # -------------- BULLETIN ----------------
    pdf_path, pdf_url, level = (None, None, None)

    if state and district:
        pdf_path, pdf_url, level = download_imd_district_bulletin(state, district)

    if pdf_path is None:
        pdf_path, pdf_url, level = download_imd_state_bulletin(state)

    print("\n📄 TRUSTED OFFICIAL BULLETIN USED (IMD Agrimet)")
    print(f"  Level : {level}")
    print(f"  PDF   : {pdf_path}")
    print(f"  URL   : {pdf_url}")

    # -------------- RAG PROCESS ----------------
    bulletin_text = extract_pdf_text(pdf_path, max_pages=10)
    vectorstore = build_vectorstore_from_text(bulletin_text)

    query, docs = retrieve_relevant_docs(vectorstore, weather, forecast, k=6)

    categorized = categorize_advisory_points(docs)
    categorized = add_weather_based_advice(weather, forecast, categorized)
    categorized = filter_by_crop(categorized, crop)

    # ---------------- PRIORITY ----------------
    top_advisories = rank_top_advisories(categorized, weather, forecast, top_n=5)

    # -------------- PRINT TOP ---------------
    print("\n" + line)
    print(" TOP IMPORTANT ADVISORIES FOR TODAY")
    print(line)

    if not top_advisories:
        print("No strong advisory points found.")
    else:
        for i, (score, cat, text) in enumerate(top_advisories, 1):
            print(f"{i}. [{cat}] {text}")

    # -------------- CATEGORY WISE --------------
    print("\n" + line)
    print(" CATEGORY-WISE FARMING ADVISORY")
    print(line)

    # Order sections (for professional output)
    section_order = [
        "Weather warnings",
        "Crop pest/disease alert",
        "Irrigation guidance",
        "Spraying / fertilizer guidance",
        "Harvest / storage / field operations"
    ]

    for section in section_order:
        points = categorized.get(section, [])
        print(f"\n🔸 {section}")
        if not points:
            print("  (No advisory found.)")
        else:
            for i, p in enumerate(points, 1):
                print(f"  {i}. {p}")

    print("\n⚠️ Disclaimer: The above chemical usage/doses are taken from official bulletin text. Farmers should confirm with local agriculture officer/extension services before application.")

    # -------------- RETURN (for API / future UI) ----------------
    return {
        "city": city,
        "crop": crop,
        "weather": weather,
        "forecast_24h": forecast,
        "state": state,
        "district": district,
        "bulletin_level": level,
        "bulletin_pdf": pdf_path,
        "bulletin_url": pdf_url,
        "rag_query": query,
        "top_advisories": top_advisories,
        "categorized_advisory": categorized
    }



In [23]:
result = smart_farm_advisory("Hyderabad")


🌾 SMART FARM ADVISORY SYSTEM | CITY: Hyderabad

🌦 REAL-TIME WEATHER (OpenWeather)
  Temperature : 28.23 °C
  Humidity    : 30 %
  Wind Speed  : 5.14 m/s
  Rain (1h)   : 0 mm
  Condition   : few clouds

⏳ NEXT 24H FORECAST SUMMARY
  Temp Range  : 19.8°C to 30.76°C
  Rain Total  : 0 mm
  Wind Max    : 3.07 m/s

📍 LOCATION MAPPING
  State   : Telangana
  District: Hyderabad

📄 TRUSTED OFFICIAL BULLETIN USED (IMD Agrimet)
  Level : STATE
  PDF   : imd_cache/Telangana_STATE_2026-02-07.pdf
  URL   : https://imdagrimet.gov.in/Services/StateBulletin.php?state=Telangana&language=English


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



 TOP IMPORTANT ADVISORIES FOR TODAY
1. [Irrigation guidance] Low humidity + no rain: Increase irrigation frequency and conserve moisture using mulching.
2. [Weather warnings] No rainfall expected in next 24 hours.
3. [Crop pest/disease alert] control, Armyworm incidence in Maize crop. Monitor the pest, if noticed,
4. [Crop pest/disease alert] Bengalgram v If the symptoms of wilt disease is seen in bengalgram, drench th base of plants with
5. [Spraying / fertilizer guidance] Blackgram. To control, spray Fipronil 5% EC @ 2 ml or Spinosad 45% SC @ 0.3 ml per

 CATEGORY-WISE FARMING ADVISORY

🔸 Weather warnings
  1. No rainfall expected in next 24 hours.

🔸 Crop pest/disease alert
  1. control, Armyworm incidence in Maize crop. Monitor the pest, if noticed,
  2. Bengalgram v If the symptoms of wilt disease is seen in bengalgram, drench th base of plants with

🔸 Irrigation guidance
  1. Low humidity + no rain: Increase irrigation frequency and conserve moisture using mulching.

🔸 Spraying 

In [27]:
def smart_farm_advisory_proof(city: str, crop: str=None):
    print("="*80)
    print(f"🌾 SMART FARM ADVISORY SYSTEM | CITY: {city}")
    print("="*80)

    # Weather
    weather = get_current_weather(city)
    forecast = get_24h_forecast_summary(city)

    print("\n🌦 REAL-TIME WEATHER (OpenWeather)")
    print(f"  Temperature : {weather['temp']} °C")
    print(f"  Humidity    : {weather['humidity']} %")
    print(f"  Wind Speed  : {weather['wind_speed']} m/s")
    print(f"  Rain (1h)   : {weather['rain_1h']} mm")
    print(f"  Condition   : {weather['description']}")

    print("\n⏳ NEXT 24H FORECAST SUMMARY")
    print(f"  Temp Range  : {forecast['temp_min_24h']}°C to {forecast['temp_max_24h']}°C")
    print(f"  Rain Total  : {forecast['rain_total_24h']} mm")
    print(f"  Wind Max    : {forecast['wind_max_24h']} m/s")

    # Location
    loc = geocode_city_openweather(city)
    state = loc["state"]
    district = reverse_geocode_district(loc["lat"], loc["lon"])

    print("\n📍 LOCATION MAPPING")
    print(f"  State   : {state}")
    print(f"  District: {district}")

    # Bulletin
    pdf_path, pdf_url, level = (None, None, None)
    if state and district:
        pdf_path, pdf_url, level = download_imd_district_bulletin(state, district)

    if pdf_path is None:
        pdf_path, pdf_url, level = download_imd_state_bulletin(state)

    print("\n📄 TRUSTED OFFICIAL BULLETIN USED (IMD Agrimet)")
    print(f"  Level : {level}")
    print(f"  PDF   : {pdf_path}")
    print(f"  URL   : {pdf_url}")

    # Knowledge base
    bulletin_text = extract_pdf_text(pdf_path, max_pages=10)
    bulletin_meta = extract_bulletin_metadata(bulletin_text)

    vectorstore = build_vectorstore_from_text(bulletin_text)

    # Retrieve + categorize
    query, docs = retrieve_relevant_docs(vectorstore, weather, forecast, k=6)

    categorized = categorize_advisory_points(docs)
    categorized = add_weather_based_advice(weather, forecast, categorized)
    categorized = filter_by_crop(categorized, crop)

    # Priority ranking
    top_advisories = rank_top_advisories(categorized, weather, forecast, top_n=5)

    # PRINT TOP PRIORITY
    print("\n" + "="*80)
    print("TOP IMPORTANT ADVISORIES FOR TODAY")
    print("="*80)
    if not top_advisories:
        print("No strong advisory points found.")
    else:
        for i, (score, cat, line) in enumerate(top_advisories, 1):
            print(f"{i}. [{cat}] {line}")

    # PRINT CATEGORY OUTPUT
    print("\n" + "="*80)
    print("CATEGORY-WISE FARMING ADVISORY")
    print("="*80)

    for section, points in categorized.items():
        print(f"\n🔸 {section}")
        if not points:
            print("  (No advisory found.)")
        else:
            for i, p in enumerate(points, 1):
                print(f"  {i}. {p}")

    # PROOF SECTION
    proof = build_proof_section(weather, forecast, pdf_url, bulletin_meta, docs)

    print("\n" + "="*80)
    print("PROOF SECTION (Why these results are trustworthy)")
    print("="*80)

    print("\n📌 Weather Proof:")
    print(f"  Source: {proof['weather_proof']['source']}")
    print(f"  Used Values: {proof['weather_proof']['live_values_used']}")

    print("\n📌 Bulletin Proof:")
    print(f"  Source: {proof['bulletin_proof']['source']}")
    print(f"  Bulletin URL: {proof['bulletin_proof']['bulletin_url']}")
    print(f"  Bulletin Number: {proof['bulletin_proof']['bulletin_number']}")
    if proof['bulletin_proof']['forecast_date_from'] and proof['bulletin_proof']['forecast_date_to']:
        print(f"  Forecast Valid: {proof['bulletin_proof']['forecast_date_from']} to {proof['bulletin_proof']['forecast_date_to']}")

    print("\n📌 RAG Evidence Snippets (Retrieved from PDF):")
    for e in proof["rag_evidence"][:4]:
        print(f"  {e['rank']}. {e['snippet']}...")

    print("\n⚠️ Disclaimer: Chemical usage/doses are taken from official bulletin text. Farmers should confirm with local agriculture officer/extension services before application.")

    return {
        "city": city,
        "crop": crop,
        "weather": weather,
        "forecast_24h": forecast,
        "state": state,
        "district": district,
        "bulletin_level": level,
        "bulletin_pdf": pdf_path,
        "bulletin_url": pdf_url,
        "bulletin_meta": bulletin_meta,
        "rag_query": query,
        "top_advisories": top_advisories,
        "categorized_advisory": categorized,
        "proof": proof
    }


In [28]:
result = smart_farm_advisory_proof("Hyderabad")
result["proof"]


🌾 SMART FARM ADVISORY SYSTEM | CITY: Hyderabad

🌦 REAL-TIME WEATHER (OpenWeather)
  Temperature : 28.23 °C
  Humidity    : 30 %
  Wind Speed  : 5.14 m/s
  Rain (1h)   : 0 mm
  Condition   : few clouds

⏳ NEXT 24H FORECAST SUMMARY
  Temp Range  : 19.8°C to 30.76°C
  Rain Total  : 0 mm
  Wind Max    : 3.07 m/s

📍 LOCATION MAPPING
  State   : Telangana
  District: Hyderabad

📄 TRUSTED OFFICIAL BULLETIN USED (IMD Agrimet)
  Level : STATE
  PDF   : imd_cache/Telangana_STATE_2026-02-07.pdf
  URL   : https://imdagrimet.gov.in/Services/StateBulletin.php?state=Telangana&language=English


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



TOP IMPORTANT ADVISORIES FOR TODAY
1. [Irrigation guidance] Low humidity + no rain: Increase irrigation frequency and conserve moisture using mulching.
2. [Weather warnings] No rainfall expected in next 24 hours.
3. [Crop pest/disease alert] control, Armyworm incidence in Maize crop. Monitor the pest, if noticed,
4. [Crop pest/disease alert] Bengalgram v If the symptoms of wilt disease is seen in bengalgram, drench th base of plants with
5. [Spraying / fertilizer guidance] Blackgram. To control, spray Fipronil 5% EC @ 2 ml or Spinosad 45% SC @ 0.3 ml per

CATEGORY-WISE FARMING ADVISORY

🔸 Weather warnings
  1. No rainfall expected in next 24 hours.

🔸 Crop pest/disease alert
  1. control, Armyworm incidence in Maize crop. Monitor the pest, if noticed,
  2. Bengalgram v If the symptoms of wilt disease is seen in bengalgram, drench th base of plants with

🔸 Irrigation guidance
  1. Low humidity + no rain: Increase irrigation frequency and conserve moisture using mulching.

🔸 Spraying / 

{'weather_proof': {'source': 'OpenWeatherMap API',
  'live_values_used': {'temp': 28.23,
   'humidity': 30,
   'wind_speed': 5.14,
   'rain_1h': 0,
   'forecast_rain_24h': 0}},
 'bulletin_proof': {'source': 'IMD GKMS / IMD Agrimet official bulletin PDF',
  'bulletin_url': 'https://imdagrimet.gov.in/Services/StateBulletin.php?state=Telangana&language=English',
  'bulletin_number': '11/2026',
  'forecast_date_from': '03-02-2026',
  'forecast_date_to': '05-02-2026'},
 'rag_evidence': [{'rank': 1,
   'snippet': 'A.4. Likely impacts of weather warnings on Agriculture ए.4.जलवाय ु चेताव7नयO का कृ=ष पर संभा=वत Pभाव Nil Livestock / Fishery Nil A.5. Realized Rainfall & distribution during last Three days from 03-02-2026 to 05-02-2026 ए.5. 03-02-2026 से 05-02-2026 तक Iपछले चार %दन= के दौरान वाTतIवक वषाV और Iवतरण 03.02.2026 04.02.2026 05.02.2026 A.6. Rainfall Fore'},
  {'rank': 2,
   'snippet': 'A.7. WEATHER FORECAST FOR THE NEXT FIVE DAYS FROM 07-02-2026 to 11-02-2026 ए.7. अगले पांच %दन= के Yलए 0